# Задача

Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:

1. InvoiceNo — номер транзакции
2. StockCode — код товара
3. Description — описание товара
4. Quantity — количество единиц товара, добавленных в заказ
5. InvoiceDate — дата транзакции 
6. UnitPrice — цена за единицу товара
7. CustomerID — id клиента
8. Country — страна, где проживает клиент


# Вопросы
1. Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). Сколько всего транзакций отменили пользователи? Каждый отдельный тип продукта считайте за отдельную транзакцию - уникальные номера считать не надо.

2. В каких заказах больше 1 товара?

3. Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. 

4. Отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам

5. Какой из продуктов добавляли в корзину чаще всего, кроме POST?

6. Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа.

In [3]:
#Загружаем библиотеку

import pandas as pd

In [4]:
# загружаем исходные данные
retail = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-evsa-22/lesson_5/5_data.csv.zip', compression = 'zip', encoding = 'ISO-8859-1')

retail_columns = retail.columns

In [5]:
# Исследуем данные

retail.dtypes
retail.shape #541909
retail = retail.drop_duplicates()
retail.shape #536641

(536641, 8)

In [6]:
#Сколько всего транзакций отменили пользователи? 

retail['InvoiceNo'].str.startswith('C').sum()

9251

In [7]:
#В каких заказах больше 1 товара?

retail[retail.Quantity > 0].count() #526054


InvoiceNo      526054
StockCode      526054
Description    525462
Quantity       526054
InvoiceDate    526054
UnitPrice      526054
CustomerID     392732
Country        526054
dtype: int64

In [8]:
#Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). 
#Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. 


qu = retail[retail.Country =='Germany'].groupby('CustomerID', as_index=False).agg({'InvoiceNo':'nunique'}).InvoiceNo.quantile(q=0.8)
germany_top  = retail[retail.Country =='Germany'].groupby('CustomerID', as_index=False).agg({'InvoiceNo':'nunique'}).query('InvoiceNo>@qu').CustomerID

In [9]:
# отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам

top_retail_germany = retail.query('CustomerID in @germany_top')

In [10]:
#Какой из продуктов добавляли в корзину чаще всего, кроме POST? 
#Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

#top_retail_germany.head(3)
top_retail_germany.groupby('StockCode', as_index=False).count().sort_values(by='InvoiceNo', ascending=False)

,StockCode,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1167,POST,219,219,219,219,219,219,219
409,22326,62,62,62,62,62,62,62
452,22423,55,55,55,55,55,55,55
411,22328,45,45,45,45,45,45,45
474,22467,37,37,37,37,37,37,37
...,...,...,...,...,...,...,...,...
743,23008,1,1,1,1,1,1,1
739,22999,1,1,1,1,1,1,1
44,20717,1,1,1,1,1,1,1
333,22119,1,1,1,1,1,1,1


In [11]:
# Создайем доп колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.
retail['Revenue'] = retail['Quantity'] * retail['UnitPrice']  


In [13]:
#Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. 
# В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа 

retail.head(5)
retail.groupby('InvoiceNo', as_index=False).agg({'Revenue':'sum'}).sort_values(by='Revenue', ascending=False).head(5).InvoiceNo.to_list()

['581483', '541431', '574941', '576365', '556444']